In [1]:
# Carbon Footprint Calculator for Maharashtra, India
# Google Colab Version
# Created: 2024

# Install required packages
!pip install pandas matplotlib ipywidgets plotly -q

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Maharashtra Carbon Footprint Calculator Loaded!")
print("=" * 60)

✅ Maharashtra Carbon Footprint Calculator Loaded!


In [2]:
class MaharashtraEmissionFactors:
    """Emission factors specific to Maharashtra, India"""

    def __init__(self):
        # Source: CEA (Central Electricity Authority) 2022-23 Report
        self.grid_emission_factor = 0.72  # kg CO2/kWh for Maharashtra grid

        # Source: PPAC (Petroleum Planning & Analysis Cell) 2023
        self.fuel_emission_factors = {
            'petrol': 2.296,      # kg CO2 per liter
            'diesel': 2.653,      # kg CO2 per liter
            'cng': 2.750,         # kg CO2 per kg
            'lpg': 1.550,         # kg CO2 per kg
            'lpg_cylinder': 22.01, # kg CO2 per 14.2kg cylinder (1.55 * 14.2)
        }

        # Source: TERI, ICAR, and Indian research
        self.diet_emissions = {
            'strict_veg': 730,       # kg CO2/year - minimal dairy
            'lacto_veg': 980,        # kg CO2/year - includes dairy
            'eggetarian': 1050,      # kg CO2/year - includes eggs
            'nonveg_occasional': 1450, # kg CO2/year - meat 1-2 times/week
            'nonveg_regular': 2100,  # kg CO2/year - meat 3-5 times/week
            'nonveg_heavy': 2850,    # kg CO2/year - daily meat
        }

        # Source: ARAI (Automotive Research Association of India)
        self.transport_factors = {
            # kg CO2 per passenger-km
            'petrol_car': 0.175,      # Assuming 13-14 km/l in city
            'diesel_car': 0.155,      # Assuming 15-16 km/l
            'cng_car': 0.125,
            'petrol_bike': 0.050,     # 40-50 km/l
            'diesel_bike': 0.045,
            'electric_car': 0.045,    # From Maharashtra grid
            'electric_bike': 0.013,
            'bus': 0.080,             # BEST/ST buses
            'auto': 0.070,            # CNG auto-rickshaw
            'train': 0.030,           # Local trains
            'metro': 0.025,           # Mumbai Metro
            'taxi': 0.130,            # App-based cabs
        }

        # Source: Maharashtra Pollution Control Board, CPCB
        self.waste_emissions = {
            'landfill': 1.05,     # kg CO2/kg waste (Deonar landfill)
            'composted': 0.12,
            'recycled': 0.20,
            'incinerated': 0.85,
        }

        # Water-related emissions (MCGM data)
        self.water_factors = {
            'treatment': 0.30,    # kg CO2/m³
            'electric_heater': 0.72,  # kg CO2/kWh
            'gas_heater': 0.198,      # kg CO2/MJ
        }

        # Consumer goods (approx values)
        self.consumption_factors = {
            'clothing': 8.0,      # kg CO2 per item (Indian garments)
            'electronics': 80.0,  # kg CO2 per smartphone
            'furniture': 120.0,   # kg CO2 per piece
            'online_shopping': 0.25,  # kg CO2 per ₹100 spent
        }

        # Travel emissions
        self.travel_factors = {
            'domestic_flight': 0.158,  # kg CO2/passenger-km (India avg)
            'train_ac': 0.040,
            'train_nonac': 0.025,
            'intercity_bus': 0.055,
        }

        # Maharashtra averages (for comparison)
        self.maharashtra_averages = {
            'per_capita': 1.6,      # tons CO2/year
            'electricity': 900,     # kg CO2/year
            'transport': 650,       # kg CO2/year
            'food': 1050,           # kg CO2/year
            'lpg': 264,             # kg CO2/year (12 cylinders)
            'waste': 150,           # kg CO2/year
            'water': 45,            # kg CO2/year
            'other': 200,           # kg CO2/year
        }

# Initialize emission factors
ef = MaharashtraEmissionFactors()

In [ ]:
def create_questionnaire():
    """Create interactive form for data collection"""

    print("📝 MAHARASHTRA CARBON FOOTPRINT QUESTIONNAIRE")
    print("=" * 60)

    # Household Information
    print("\n🏠 HOUSEHOLD INFORMATION")
    household_size = widgets.IntSlider(
        value=4,
        min=1,
        max=20,
        step=1,
        description='Family members:',
        continuous_update=False
    )

    location = widgets.Dropdown(
        options=['Mumbai', 'Pune', 'Nagpur', 'Nashik', 'Aurangabad', 'Other Urban', 'Rural'],
        value='Mumbai',
        description='Location:'
    )

    # Electricity
    print("\n⚡ ELECTRICITY CONSUMPTION")
    electricity_bill = widgets.FloatSlider(
        value=2000,
        min=0,
        max=20000,
        step=100,
        description='Monthly bill (₹):',
        continuous_update=False
    )

    # Transportation
    print("\n🚗 TRANSPORTATION")
    commute_mode = widgets.Dropdown(
        options=[
            'Train (local)',
            'Bus',
            'Car (petrol)',
            'Car (diesel)',
            'Car (CNG)',
            '2-wheeler (petrol)',
            '2-wheeler (electric)',
            'Auto-rickshaw',
            'Metro',
            'Walk/Cycle'
        ],
        value='Train (local)',
        description='Primary commute:'
    )

    commute_distance = widgets.FloatSlider(
        value=10,
        min=0,
        max=100,
        step=1,
        description='One-way distance (km):',
        continuous_update=False
    )

    work_days = widgets.IntSlider(
        value=22,
        min=0,
        max=31,
        step=1,
        description='Work days/month:',
        continuous_update=False
    )

    # LPG Consumption
    print("\n🔥 DOMESTIC FUELS")
    lpg_cylinders = widgets.FloatSlider(
        value=1,
        min=0,
        max=5,
        step=0.5,
        description='LPG cylinders/month:',
        continuous_update=False
    )

    # Dietary Habits
    print("\n🍛 DIETARY HABITS")
    diet_type = widgets.Dropdown(
        options=[
            'Strict Vegetarian (no dairy/eggs)',
            'Lacto Vegetarian (with dairy)',
            'Eggetarian (with eggs)',
            'Non-vegetarian (occasional meat)',
            'Non-vegetarian (regular meat)',
            'Non-vegetarian (heavy meat)'
        ],
        value='Eggetarian (with eggs)',
        description='Diet type:'
    )

    rice_frequency = widgets.Dropdown(
        options=['Daily', '5-6 days/week', '3-4 days/week', '1-2 days/week', 'Rarely'],
        value='Daily',
        description='Rice consumption:'
    )

    # Water Consumption
    print("\n💧 WATER USAGE")
    water_bill = widgets.FloatSlider(
        value=500,
        min=0,
        max=5000,
        step=50,
        description='Monthly water bill (₹):',
        continuous_update=False
    )

    geyser_type = widgets.Dropdown(
        options=['Electric', 'Gas', 'Solar', 'None'],
        value='Electric',
        description='Water heater:'
    )

    # Waste Generation
    print("\n🗑️ WASTE MANAGEMENT")
    waste_kg = widgets.FloatSlider(
        value=1.5,
        min=0,
        max=10,
        step=0.1,
        description='Waste per person/day (kg):',
        continuous_update=False
    )

    waste_segregation = widgets.Dropdown(
        options=['Yes', 'No'],
        value='No',
        description='Segregate waste:'
    )

    # Air Travel
    print("\n✈️ AIR TRAVEL")
    domestic_flights = widgets.IntSlider(
        value=1,
        min=0,
        max=50,
        step=1,
        description='Domestic flights/year:',
        continuous_update=False
    )

    # Shopping Habits
    print("\n🛒 SHOPPING HABITS")
    online_shopping = widgets.FloatSlider(
        value=2000,
        min=0,
        max=50000,
        step=500,
        description='Online shopping/month (₹):',
        continuous_update=False
    )

    clothing_items = widgets.IntSlider(
        value=4,
        min=0,
        max=50,
        step=1,
        description='Clothing items purchased/year:',
        continuous_update=False
    )

    # Display all widgets
    display(household_size, location)
    display(electricity_bill)
    display(commute_mode, commute_distance, work_days)
    display(lpg_cylinders)
    display(diet_type, rice_frequency)
    display(water_bill, geyser_type)
    display(waste_kg, waste_segregation)
    display(domestic_flights)
    display(online_shopping, clothing_items)

    # Calculate button
    calculate_btn = widgets.Button(
        description='CALCULATE FOOTPRINT',
        button_style='success',
        icon='calculator'
    )

    output = widgets.Output()

    def on_calculate_click(b):
        with output:
            clear_output()

            # Collect all responses
            user_data = {
                'household_size': household_size.value,
                'location': location.value,
                'electricity_bill': electricity_bill.value,
                'commute_mode': commute_mode.value,
                'commute_distance': commute_distance.value,
                'work_days': work_days.value,
                'lpg_cylinders': lpg_cylinders.value,
                'diet_type': diet_type.value,
                'rice_frequency': rice_frequency.value,
                'water_bill': water_bill.value,
                'geyser_type': geyser_type.value,
                'waste_kg': waste_kg.value,
                'waste_segregation': waste_segregation.value,
                'domestic_flights': domestic_flights.value,
                'online_shopping': online_shopping.value,
                'clothing_items': clothing_items.value,
            }

            # Calculate footprint
            calculator = CarbonFootprintCalculator(ef)
            results = calculator.calculate_footprint(user_data)

            # Display results
            display_results(results)

    calculate_btn.on_click(on_calculate_click)
    display(calculate_btn)
    display(output)

# Create the questionnaire
create_questionnaire()

📝 MAHARASHTRA CARBON FOOTPRINT QUESTIONNAIRE

🏠 HOUSEHOLD INFORMATION

⚡ ELECTRICITY CONSUMPTION

🚗 TRANSPORTATION

🔥 DOMESTIC FUELS

🍛 DIETARY HABITS

💧 WATER USAGE

🗑️ WASTE MANAGEMENT

✈️ AIR TRAVEL

🛒 SHOPPING HABITS


IntSlider(value=4, continuous_update=False, description='Family members:', max=20, min=1)

Dropdown(description='Location:', options=('Mumbai', 'Pune', 'Nagpur', 'Nashik', 'Aurangabad', 'Other Urban', …

FloatSlider(value=2000.0, continuous_update=False, description='Monthly bill (₹):', max=20000.0, step=100.0)

Dropdown(description='Primary commute:', options=('Train (local)', 'Bus', 'Car (petrol)', 'Car (diesel)', 'Car…

...

IntSlider(value=1, continuous_update=False, description='Domestic flights/year:', max=50)

FloatSlider(value=2000.0, continuous_update=False, description='Online shopping/month (₹):', max=50000.0, step…

IntSlider(value=4, continuous_update=False, description='Clothing items purchased/year:', max=50)

Button(button_style='success', description='CALCULATE FOOTPRINT', icon='calculator', style=ButtonStyle())

Output()

In [4]:
class CarbonFootprintCalculator:
    """Main calculator class"""

    def __init__(self, emission_factors):
        self.ef = emission_factors
        self.results = {}

    def calculate_footprint(self, user_data):
        """Calculate total carbon footprint"""

        # Initialize results dictionary
        self.results = {
            'user_data': user_data,
            'breakdown': {},
            'total': 0,
            'per_capita': 0,
            'vs_average': 0
        }

        # 1. ELECTRICITY
        self._calculate_electricity(user_data)

        # 2. TRANSPORTATION
        self._calculate_transportation(user_data)

        # 3. DOMESTIC FUELS (LPG)
        self._calculate_domestic_fuels(user_data)

        # 4. DIET
        self._calculate_diet(user_data)

        # 5. WATER
        self._calculate_water(user_data)

        # 6. WASTE
        self._calculate_waste(user_data)

        # 7. AIR TRAVEL
        self._calculate_air_travel(user_data)

        # 8. CONSUMER GOODS
        self._calculate_consumer_goods(user_data)

        # Calculate totals
        self._calculate_totals()

        # Generate recommendations
        self._generate_recommendations()

        return self.results

    def _calculate_electricity(self, data):
        """Calculate electricity emissions"""
        # Convert bill to kWh (approx 8₹/kWh in Maharashtra)
        monthly_kwh = data['electricity_bill'] / 8
        annual_kwh = monthly_kwh * 12

        # Calculate CO2
        annual_co2 = annual_kwh * self.ef.grid_emission_factor

        # Adjust for household size
        per_capita_co2 = annual_co2 / data['household_size']

        self.results['breakdown']['electricity'] = {
            'kwh_per_year': annual_kwh,
            'co2_kg': per_capita_co2,
            'description': f"{monthly_kwh:.1f} kWh/month, {annual_kwh:.0f} kWh/year"
        }

    def _calculate_transportation(self, data):
        """Calculate transportation emissions"""
        # Map commute mode to emission factor
        mode_map = {
            'Train (local)': 'train',
            'Bus': 'bus',
            'Car (petrol)': 'petrol_car',
            'Car (diesel)': 'diesel_car',
            'Car (CNG)': 'cng_car',
            '2-wheeler (petrol)': 'petrol_bike',
            '2-wheeler (electric)': 'electric_bike',
            'Auto-rickshaw': 'auto',
            'Metro': 'metro',
            'Walk/Cycle': 0
        }

        mode = data['commute_mode']
        if mode == 'Walk/Cycle':
            transport_co2 = 0
        else:
            factor_key = mode_map[mode]
            # Daily commute emissions (round trip)
            daily_emissions = data['commute_distance'] * 2 * self.ef.transport_factors[factor_key]
            # Monthly emissions
            monthly_emissions = daily_emissions * data['work_days']
            # Annual emissions
            transport_co2 = monthly_emissions * 12

        self.results['breakdown']['transport'] = {
            'co2_kg': transport_co2,
            'description': f"{data['commute_distance']}km {mode}, {data['work_days']} days/month"
        }

    def _calculate_domestic_fuels(self, data):
        """Calculate LPG emissions"""
        annual_cylinders = data['lpg_cylinders'] * 12
        total_co2 = annual_cylinders * self.ef.fuel_emission_factors['lpg_cylinder']
        per_capita_co2 = total_co2 / data['household_size']

        self.results['breakdown']['lpg'] = {
            'co2_kg': per_capita_co2,
            'description': f"{data['lpg_cylinders']} cylinders/month"
        }

    def _calculate_diet(self, data):
        """Calculate dietary emissions"""
        # Base diet emissions
        diet_map = {
            'Strict Vegetarian (no dairy/eggs)': 'strict_veg',
            'Lacto Vegetarian (with dairy)': 'lacto_veg',
            'Eggetarian (with eggs)': 'eggetarian',
            'Non-vegetarian (occasional meat)': 'nonveg_occasional',
            'Non-vegetarian (regular meat)': 'nonveg_regular',
            'Non-vegetarian (heavy meat)': 'nonveg_heavy'
        }

        diet_key = diet_map[data['diet_type']]
        diet_co2 = self.ef.diet_emissions[diet_key]

        # Rice consumption adjustment
        rice_adjustment = {
            'Daily': 150,
            '5-6 days/week': 120,
            '3-4 days/week': 80,
            '1-2 days/week': 40,
            'Rarely': 0
        }
        diet_co2 += rice_adjustment[data['rice_frequency']]

        self.results['breakdown']['diet'] = {
            'co2_kg': diet_co2,
            'description': f"{data['diet_type']}, Rice: {data['rice_frequency']}"
        }

    def _calculate_water(self, data):
        """Calculate water-related emissions"""
        # Water treatment (approx 10₹/m³ in Maharashtra)
        monthly_water_m3 = data['water_bill'] / 10
        annual_treatment_co2 = monthly_water_m3 * 12 * self.ef.water_factors['treatment']

        # Water heating (assume 30 liters/day, 30°C rise)
        if data['geyser_type'] == 'Electric':
            # 30L * 30°C * 0.001163 kWh/L°C = 1.05 kWh/day
            daily_heating_kwh = 1.05
            annual_heating_co2 = daily_heating_kwh * 365 * self.ef.water_factors['electric_heater']
        elif data['geyser_type'] == 'Gas':
            annual_heating_co2 = 50  # Approx value
        else:
            annual_heating_co2 = 0

        total_water_co2 = (annual_treatment_co2 + annual_heating_co2) / data['household_size']

        self.results['breakdown']['water'] = {
            'co2_kg': total_water_co2,
            'description': f"Bill: ₹{data['water_bill']}/month, Heater: {data['geyser_type']}"
        }

    def _calculate_waste(self, data):
        """Calculate waste emissions"""
        daily_waste_kg = data['waste_kg'] * data['household_size']

        # Determine disposal method based on segregation
        if data['waste_segregation'] == 'Yes':
            # Assume 50% composted, 30% recycled, 20% landfill
            compost_co2 = daily_waste_kg * 0.5 * self.ef.waste_emissions['composted']
            recycle_co2 = daily_waste_kg * 0.3 * self.ef.waste_emissions['recycled']
            landfill_co2 = daily_waste_kg * 0.2 * self.ef.waste_emissions['landfill']
            total_co2 = (compost_co2 + recycle_co2 + landfill_co2) * 365
        else:
            # All to landfill
            total_co2 = daily_waste_kg * self.ef.waste_emissions['landfill'] * 365

        per_capita_co2 = total_co2 / data['household_size']

        self.results['breakdown']['waste'] = {
            'co2_kg': per_capita_co2,
            'description': f"{data['waste_kg']} kg/person/day, Segregation: {data['waste_segregation']}"
        }

    def _calculate_air_travel(self, data):
        """Calculate air travel emissions"""
        # Average domestic flight in India: 1000 km
        avg_flight_distance = 1000
        annual_co2 = data['domestic_flights'] * avg_flight_distance * 2 * self.ef.travel_factors['domestic_flight']

        self.results['breakdown']['air_travel'] = {
            'co2_kg': annual_co2,
            'description': f"{data['domestic_flights']} domestic flights/year"
        }

    def _calculate_consumer_goods(self, data):
        """Calculate consumer goods emissions"""
        # Online shopping
        online_co2 = (data['online_shopping'] * 12 / 100) * self.ef.consumption_factors['online_shopping']

        # Clothing
        clothing_co2 = data['clothing_items'] * self.ef.consumption_factors['clothing']

        total_co2 = online_co2 + clothing_co2

        self.results['breakdown']['consumer_goods'] = {
            'co2_kg': total_co2,
            'description': f"Shopping: ₹{data['online_shopping']}/month, {data['clothing_items']} clothes/year"
        }

    def _calculate_totals(self):
        """Calculate total footprint"""
        total_co2 = 0
        for category in self.results['breakdown'].values():
            total_co2 += category['co2_kg']

        self.results['total_kg'] = total_co2
        self.results['total_tons'] = total_co2 / 1000

        # Per capita (already calculated per capita for shared resources)
        self.results['per_capita_kg'] = total_co2
        self.results['per_capita_tons'] = total_co2 / 1000

        # Compare with Maharashtra average
        maharashtra_avg = self.ef.maharashtra_averages['per_capita'] * 1000  # Convert to kg
        self.results['vs_average_kg'] = total_co2 - maharashtra_avg
        self.results['vs_average_percent'] = (total_co2 / maharashtra_avg - 1) * 100

    def _generate_recommendations(self):
        """Generate personalized recommendations"""
        recommendations = []

        data = self.results['user_data']
        breakdown = self.results['breakdown']

        # Electricity recommendations
        if breakdown['electricity']['co2_kg'] > self.ef.maharashtra_averages['electricity']:
            recommendations.append({
                'category': 'Electricity',
                'action': 'Switch to LED bulbs and use appliances during off-peak hours',
                'savings': 'Up to 20% reduction in electricity bill'
            })

        # Transport recommendations
        if breakdown['transport']['co2_kg'] > self.ef.maharashtra_averages['transport']:
            if 'Car' in data['commute_mode']:
                recommendations.append({
                    'category': 'Transport',
                    'action': 'Use public transport or carpool 2 days/week',
                    'savings': 'Reduce transport emissions by 30%'
                })

        # LPG recommendations
        if breakdown['lpg']['co2_kg'] > self.ef.maharashtra_averages['lpg']:
            recommendations.append({
                'category': 'Cooking',
                'action': 'Use pressure cooker and maintain stove properly',
                'savings': 'Save 1 LPG cylinder every 2 months'
            })

        # Diet recommendations
        if 'Non-vegetarian' in data['diet_type']:
            recommendations.append({
                'category': 'Diet',
                'action': 'Have 2 meat-free days per week',
                'savings': 'Reduce food emissions by 15%'
            })

        # Waste recommendations
        if data['waste_segregation'] == 'No':
            recommendations.append({
                'category': 'Waste',
                'action': 'Start segregating wet and dry waste',
                'savings': 'Reduce landfill emissions by 50%'
            })

        # General recommendations for Maharashtra
        recommendations.extend([
            {
                'category': 'Maharashtra Specific',
                'action': 'Install solar water heater (great for Maharashtra climate)',
                'savings': 'Save ₹3000-5000/year on electricity'
            },
            {
                'category': 'Water',
                'action': 'Install rainwater harvesting (especially in drought-prone areas)',
                'savings': 'Reduce water bill by 30%'
            }
        ])

        self.results['recommendations'] = recommendations

In [5]:
def display_results(results):
    """Display results with visualizations"""

    print("\n" + "=" * 60)
    print("📊 CARBON FOOTPRINT RESULTS")
    print("=" * 60)

    # Summary statistics
    print(f"\n📍 Location: {results['user_data']['location']}")
    print(f"👨‍👩‍👧‍👦 Household Size: {results['user_data']['household_size']}")
    print(f"📅 Calculated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}")

    print("\n" + "=" * 60)
    print("📈 FOOTPRINT SUMMARY")
    print("=" * 60)

    total_tons = results['total_tons']
    avg_tons = ef.maharashtra_averages['per_capita']
    difference = results['vs_average_kg'] / 1000

    print(f"\n🏆 YOUR ANNUAL CARBON FOOTPRINT: {total_tons:.2f} tons CO₂")
    print(f"📊 MAHARASHTRA AVERAGE: {avg_tons:.2f} tons CO₂ per person")

    if difference > 0:
        print(f"🔴 YOU ARE {abs(difference):.2f} tons ABOVE the Maharashtra average")
        print(f"   That's {results['vs_average_percent']:.1f}% higher than average")
    else:
        print(f"🟢 YOU ARE {abs(difference):.2f} tons BELOW the Maharashtra average")
        print(f"   That's {abs(results['vs_average_percent']):.1f}% lower than average")

    print(f"\n🌍 GLOBAL COMPARISON:")
    print(f"   • India average: ~1.9 tons CO₂ per person")
    print(f"   • Global average: ~4.8 tons CO₂ per person")
    print(f"   • Climate target: <2.0 tons CO₂ per person by 2030")

    # Create breakdown DataFrame
    breakdown_data = []
    for category, data in results['breakdown'].items():
        breakdown_data.append({
            'Category': category.replace('_', ' ').title(),
            'CO₂ (kg/year)': data['co2_kg'],
            'CO₂ (tons/year)': data['co2_kg'] / 1000,
            'Description': data['description']
        })

    df_breakdown = pd.DataFrame(breakdown_data)
    df_breakdown = df_breakdown.sort_values('CO₂ (kg/year)', ascending=False)

    print("\n" + "=" * 60)
    print("📊 DETAILED BREAKDOWN")
    print("=" * 60)
    display(df_breakdown.style.background_gradient(subset=['CO₂ (kg/year)'], cmap='Reds'))

    # Visualizations
    print("\n" + "=" * 60)
    print("📊 VISUALIZATIONS")
    print("=" * 60)

    # Create plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Maharashtra Carbon Footprint Analysis', fontsize=16, fontweight='bold')

    # 1. Pie chart of breakdown
    categories = df_breakdown['Category']
    values = df_breakdown['CO₂ (kg/year)']

    ax1 = axes[0, 0]
    colors = plt.cm.Set3(np.linspace(0, 1, len(categories)))
    wedges, texts, autotexts = ax1.pie(values, labels=categories, autopct='%1.1f%%',
                                      colors=colors, startangle=90)
    ax1.set_title('Contribution by Category', fontweight='bold')

    # 2. Bar chart comparison with averages
    ax2 = axes[0, 1]
    user_values = []
    avg_values = []

    for idx, row in df_breakdown.iterrows():
        category_key = row['Category'].lower().replace(' ', '_')
        if category_key in ef.maharashtra_averages:
            user_values.append(row['CO₂ (kg/year)'])
            avg_values.append(ef.maharashtra_averages[category_key])

    x = np.arange(len(user_values))
    width = 0.35
    ax2.bar(x - width/2, user_values, width, label='Your Footprint', color='#FF6B6B')
    ax2.bar(x + width/2, avg_values, width, label='Maharashtra Average', color='#4ECDC4')
    ax2.set_xlabel('Category')
    ax2.set_ylabel('CO₂ (kg/year)')
    ax2.set_title('Comparison with Maharashtra Average', fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels([cat[:10] for cat in categories[:len(user_values)]], rotation=45)
    ax2.legend()

    # 3. Progress towards climate goals
    ax3 = axes[1, 0]
    goals = {
        'Your Footprint': results['total_kg'] / 1000,
        'Maharashtra Avg': ef.maharashtra_averages['per_capita'],
        'India Avg': 1.9,
        'Global Avg': 4.8,
        '2030 Target': 2.0,
        '2050 Target': 0.5
    }

    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3']
    ax3.barh(list(goals.keys()), list(goals.values()), color=colors)
    ax3.set_xlabel('Tons CO₂ per year')
    ax3.set_title('Progress Towards Climate Goals', fontweight='bold')
    ax3.axvline(x=2.0, color='red', linestyle='--', alpha=0.5, label='Paris Agreement Target')

    # 4. Monthly trend (simulated)
    ax4 = axes[1, 1]
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # Simulate seasonal variation (higher in summer for AC, festivals)
    base_monthly = results['total_kg'] / 12
    seasonal_factor = [1.0, 0.9, 0.8, 1.1, 1.3, 1.4,
                       1.2, 1.1, 1.0, 0.9, 1.0, 1.1]  # Higher in summer
    monthly_values = [base_monthly * f for f in seasonal_factor]

    ax4.plot(months, monthly_values, marker='o', color='#FF6B6B', linewidth=2)
    ax4.fill_between(months, monthly_values, alpha=0.3, color='#FF6B6B')
    ax4.set_xlabel('Month')
    ax4.set_ylabel('CO₂ (kg)')
    ax4.set_title('Estimated Monthly Footprint (with seasonality)', fontweight='bold')
    ax4.grid(True, alpha=0.3)

    # Highlight high consumption months
    ax4.axhline(y=base_monthly, color='gray', linestyle='--', alpha=0.5, label='Monthly Average')
    ax4.fill_between(months, base_monthly, monthly_values, where=np.array(monthly_values) > base_monthly,
                     color='red', alpha=0.2, label='Above Average')

    plt.tight_layout()
    plt.show()

    # Recommendations
    print("\n" + "=" * 60)
    print("💡 PERSONALIZED RECOMMENDATIONS FOR MAHARASHTRA")
    print("=" * 60)

    if 'recommendations' in results:
        for i, rec in enumerate(results['recommendations'], 1):
            print(f"\n{i}. {rec['category'].upper()}")
            print(f"   Action: {rec['action']}")
            print(f"   Expected Savings: {rec['savings']}")

    # Impact visualization
    print("\n" + "=" * 60)
    print("🌳 ENVIRONMENTAL IMPACT")
    print("=" * 60)

    # Calculate equivalent trees needed
    # One tree absorbs ~21 kg CO2 per year
    trees_needed = results['total_kg'] / 21

    # Calculate equivalent car kilometers
    # Petrol car emits ~0.175 kg CO2 per km
    car_km = results['total_kg'] / 0.175

    print(f"\n🌳 To offset your carbon footprint, you would need:")
    print(f"   • {trees_needed:.0f} trees planted and maintained for 10 years")
    print(f"   • Or avoid driving {car_km:.0f} km in a petrol car")
    print(f"   • Or save {results['total_kg']/0.72:.0f} kWh of electricity")

    # Maharashtra-specific suggestions
    print(f"\n📍 MAHARASHTRA-SPECIFIC ACTIONS:")
    print(f"   • Use Mumbai Local/Metro instead of cars for commute")
    print(f"   • Install solar panels (good ROI in Maharashtra)")
    print(f"   • Harvest rainwater (important in drought-prone regions)")
    print(f"   • Support Maharashtra's green energy initiatives")

    # Save results option
    print("\n" + "=" * 60)
    print("💾 EXPORT RESULTS")
    print("=" * 60)

    # Create export button
    export_btn = widgets.Button(
        description='📥 Export Results to CSV',
        button_style='info',
        icon='download'
    )

    output_export = widgets.Output()

    def on_export_click(b):
        with output_export:
            clear_output()

            # Create export DataFrame
            export_data = {
                'timestamp': [pd.Timestamp.now()],
                'location': [results['user_data']['location']],
                'household_size': [results['user_data']['household_size']],
                'total_co2_kg': [results['total_kg']],
                'total_co2_tons': [results['total_tons']],
                'vs_maharashtra_avg_kg': [results['vs_average_kg']],
                'vs_maharashtra_percent': [results['vs_average_percent']],
            }

            # Add category breakdown
            for category, data in results['breakdown'].items():
                export_data[f'{category}_kg'] = [data['co2_kg']]

            df_export = pd.DataFrame(export_data)

            # Save to CSV
            filename = f"maharashtra_carbon_footprint_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df_export.to_csv(filename, index=False)

            print(f"✅ Results saved to: {filename}")
            print(f"📊 You can download this file from Colab's file browser")

            # Display download link
            from IPython.display import FileLink
            display(FileLink(filename))

    export_btn.on_click(on_export_click)
    display(export_btn)
    display(output_export)

    print("\n" + "=" * 60)
    print("♻️ KEEP CALCULATING, KEEP REDUCING!")
    print("=" * 60)
    print("\nCalculate again with different inputs to see how changes affect your footprint!")

In [6]:
# Add comparison with other states
def compare_with_other_states(your_footprint):
    """Compare with other Indian states"""

    state_emissions = {
        'Maharashtra': 1.60,
        'Delhi': 2.10,
        'Tamil Nadu': 1.45,
        'Gujarat': 1.70,
        'Karnataka': 1.35,
        'Uttar Pradesh': 0.95,
        'Rajasthan': 1.20,
        'West Bengal': 1.10,
        'Kerala': 1.30,
        'India Average': 1.90,
        'Global Average': 4.80
    }

    fig, ax = plt.subplots(figsize=(10, 6))

    states = list(state_emissions.keys())
    values = list(state_emissions.values())

    # Create color array (highlight Maharashtra and user)
    colors = ['#CCCCCC' for _ in states]
    colors[states.index('Maharashtra')] = '#4ECDC4'  # Maharashtra
    colors[states.index('India Average')] = '#45B7D1'  # India average
    colors[states.index('Global Average')] = '#96CEB4'  # Global average

    bars = ax.barh(states, values, color=colors)
    ax.axvline(x=your_footprint, color='#FF6B6B', linewidth=3,
               label=f'Your Footprint: {your_footprint:.2f} tons')

    ax.set_xlabel('Tons CO₂ per capita per year')
    ax.set_title('Comparison with Other States and Averages', fontweight='bold')
    ax.legend()

    # Add value labels
    for i, (state, value) in enumerate(zip(states, values)):
        ax.text(value + 0.05, i, f'{value:.2f}', va='center', fontweight='bold')

    plt.tight_layout()
    plt.show()

# Add scenario analysis
def analyze_scenarios(base_results):
    """Analyze different scenarios"""

    print("\n" + "=" * 60)
    print("🔮 SCENARIO ANALYSIS")
    print("=" * 60)

    scenarios = {
        'Current': base_results['total_tons'],
        'Electricity -20%': base_results['total_tons'] * 0.95,
        'Transport -30%': base_results['total_tons'] * 0.93,
        'Vegetarian Diet': base_results['total_tons'] * 0.90,
        'All Actions Combined': base_results['total_tons'] * 0.80,
        'Climate Target': 2.0
    }

    fig, ax = plt.subplots(figsize=(10, 6))

    scenario_names = list(scenarios.keys())
    scenario_values = list(scenarios.values())

    colors = ['#FF6B6B', '#FECA57', '#FF9FF3', '#54A0FF', '#00D2D3', '#1DD1A1']
    bars = ax.bar(scenario_names, scenario_values, color=colors)

    ax.set_ylabel('Tons CO₂ per year')
    ax.set_title('Impact of Different Actions on Your Carbon Footprint', fontweight='bold')
    ax.set_xticklabels(scenario_names, rotation=45, ha='right')

    # Add value labels
    for bar, value in zip(bars, scenario_values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                f'{value:.2f}', ha='center', fontweight='bold')

    plt.tight_layout()
    plt.show()

    print("\n💰 ESTIMATED ANNUAL SAVINGS:")
    print(f"   • Electricity reduction: Save ₹{base_results['user_data']['electricity_bill'] * 12 * 0.2:.0f}/year")
    print(f"   • Transport reduction: Save ₹{5000:.0f}/year (fuel + maintenance)")
    print(f"   • Combined actions: Save ₹{15000:.0f}+/year")

# Main execution cell
print("\n🎯 Ready to calculate your carbon footprint!")
print("👉 Fill in the questionnaire above and click 'CALCULATE FOOTPRINT'")
print("\n💡 Tip: Be as accurate as possible for the best results!")


🎯 Ready to calculate your carbon footprint!
👉 Fill in the questionnaire above and click 'CALCULATE FOOTPRINT'

💡 Tip: Be as accurate as possible for the best results!


In [7]:
# Simple calculator for quick estimates
def quick_calculator():
    """Quick carbon footprint calculator"""

    print("⚡ QUICK CARBON FOOTPRINT ESTIMATOR")
    print("=" * 50)

    # Simple inputs
    electricity = float(input("Monthly electricity bill (₹): ") or "2000")
    commute = input("Primary commute (car/bus/train/bike): ").lower()
    diet = input("Diet (veg/nonveg): ").lower()
    flights = int(input("Domestic flights per year: ") or "2")

    # Quick calculation
    # Electricity
    monthly_kwh = electricity / 8
    annual_co2_elec = monthly_kwh * 12 * ef.grid_emission_factor

    # Transport
    transport_map = {'car': 1000, 'bus': 400, 'train': 300, 'bike': 200}
    annual_co2_trans = transport_map.get(commute, 500)

    # Diet
    diet_map = {'veg': 900, 'nonveg': 1800}
    annual_co2_diet = diet_map.get(diet, 1200)

    # Flights
    annual_co2_flights = flights * 1000 * 2 * ef.travel_factors['domestic_flight']

    # Total
    total_kg = annual_co2_elec + annual_co2_trans + annual_co2_diet + annual_co2_flights
    total_tons = total_kg / 1000

    print("\n" + "=" * 50)
    print(f"📊 QUICK ESTIMATE: {total_tons:.1f} tons CO₂/year")
    print(f"📈 Maharashtra Average: 1.6 tons CO₂/year")

    if total_tons > 1.6:
        print(f"🔴 {total_tons - 1.6:.1f} tons above average")
    else:
        print(f"🟢 {1.6 - total_tons:.1f} tons below average")

    return total_tons

# Uncomment to run quick calculator
# quick_result = quick_calculator()